In [3]:
# Imports
import os as os
import cv2 as cv
import numpy as np
from matplotlib import pyplot as plt
from scipy import ndimage
import tensorflow as tf
from tensorflow import keras
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
import random
from keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout, BatchNormalization, LeakyReLU
from keras.models import Sequential
from keras.callbacks import ModelCheckpoint
import numpy as np
from keras.models import load_model
from sklearn.metrics import mean_squared_error, mean_absolute_percentage_error

functions to extract age and group by age

In [1]:
def extract_age(filename):
    return int(filename.split('_')[0].split('\\')[-1])

In [2]:
def check_which_group_age(age):
    if 1 <= age <= 2:
        return 0
    elif 3 <= age <= 10:
        return 1
    elif 11 <= age <= 20:
        return 2
    elif 21 <= age <= 30:
        return 3
    elif 31 <= age <= 45:
        return 4
    elif 46 <= age <= 60:
        return 5
    else:
        return 6

load images from the path and preprocess them

In [8]:
def preprocess_image(filelist, max_images_per_group=50):
    images = []
    labels = []
    group_to_filepaths = {}  # Dictionary to hold file paths for each age group

    # Group the file paths by age group
    for filepath in filelist:
        age = extract_age(filepath)
        if age > 80:
            age = 80
        age_group = check_which_group_age(age)
        if age_group not in group_to_filepaths:
            group_to_filepaths[age_group] = []
        group_to_filepaths[age_group].append(filepath)

    # Shuffle and truncate lists of file paths for each age group
    for group, paths in group_to_filepaths.items():
        random.shuffle(paths)
        group_to_filepaths[group] = paths[:max_images_per_group]
    
    for i in group_to_filepaths.keys():
        print(len(group_to_filepaths[i]))
    # Process the images
    for group, paths in group_to_filepaths.items():
        for filepath in paths:
            age = extract_age(filepath)
            if age > 80:
                age = 80
            img_org = cv.imread(filepath)

            # Convert the image to grayscale
            gray = cv.cvtColor(img_org, cv.COLOR_RGB2GRAY)

            # Apply Canny edge detection to find edges
            edges = cv.Canny(gray, 100, 200)

            # Perform dilation to thicken the edges
            kernel = np.ones((3, 3), np.uint8)
            thick_edges = cv.dilate(edges, kernel, iterations=2)
            img_combined = cv.addWeighted(img_org, 0.7, cv.cvtColor(thick_edges, cv.COLOR_GRAY2RGB), 0.3, 0)
            img_combined = cv.resize(img_combined, (200, 200)) 
            img_combined = img_combined.astype(np.float32) / 255.0

            images.append(img_combined)
            labels.append(age)
    
    return np.array(images), np.array(labels)


input here your path to folder with images and amount of photos for each group of age

In [15]:
# Load and preprocess images
dirIn = r'C:\Users\Bartus\Desktop\UTKFace' # change path to your folder
all_files = [os.path.join(dirIn, filename) for filename in os.listdir(dirIn)]
X, Y = preprocess_image(all_files, 50) # chgange second param if you want more photos for each group
print(len(X), len(Y))

400
400
400
400
400
400
400
2800 2800


In [16]:
def calculate_rmse(y_true, y_pred):
    return np.sqrt(mean_squared_error(y_true, y_pred))

def calculate_mape(y_true, y_pred):
    return mean_absolute_percentage_error(y_true, y_pred)

# Load the best saved model
best_model = load_model('best_model11.h5')

# Evaluate the best model on the test set
test_loss, test_mae = best_model.evaluate(X, Y)

# Predict using the best model on the test set
y_pred = best_model.predict(X)

# Calculate additional metrics
test_rmse = calculate_rmse(Y, y_pred)
test_mape = calculate_mape(Y, y_pred)

print("Test Loss (MSE):", test_loss)
print("Test MAE:", test_mae)
print("Test RMSE:", test_rmse)
print("Test MAPE:", test_mape)


88/88 [==============================] - 7s 76ms/step
Test Loss (MSE): 57.86164474487305
Test MAE: 5.30052375793457
Test RMSE: 7.606683474184374
Test MAPE: 0.4543184362527442
